In [1]:
import pandas as pd
import requests
import numpy as np
import datetime as datetime
from bs4 import BeautifulSoup
import json

import time
import re
import pickle

In [2]:
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True)

In [3]:
#meu id
id_str = "130441299"
id = 130441299

## Antes do pickle

In [8]:
#url_matches
url_perfil= "https://api.opendota.com/api/players/"+id_str
#Pegar os dados do meu perfil e transformar request em list
perfil = json.loads(requests.get(url_perfil).text)
with open("perfil.txt", "wb") as fp:   #Pickling
    pickle.dump(perfil, fp)

In [31]:
#Pegar os dados dos heroes e transformar request em list
heroes = json.loads(requests.get("https://api.opendota.com/api/heroStats").text)
with open("heroes.txt", "wb") as fp:   #Pickling
    pickle.dump(heroes, fp)

In [33]:
#Pegar todos os ID das minhas partidas e transformar request em list
list_matches = json.loads(requests.get(url_perfil+"/matches").text)
with open("list_matches.txt", "wb") as fp:   #Pickling
    pickle.dump(list_matches, fp)

In [34]:
#pegar os id das partidas com meu id da steam
matches_id = []
for m in range(0,len(list_matches)):
    matches_id.append(str(list_matches[m]['match_id']))

In [35]:
list_matches_detalhadas = []

for i in range(0,len(matches_id)):
    url_matches_detalhadas= "https://api.opendota.com/api/matches/"+matches_id[i]
    #pegar os detalhas da minha performance. As informações de Players pega todos os players da partida, então tive que fazer uma condição
    # para comparar o player com o meu id, assim eu pego apenas as minhas informações
    matches_detalhadas = json.loads(requests.get(url_matches_detalhadas).text)
    list_matches_detalhadas.append(matches_detalhadas)

In [36]:
with open("list_detailed_match.txt", "wb") as fp:   #Pickling
    pickle.dump(list_matches_detalhadas, fp)

## Depois do pickle

In [8]:
with open("list_detailed_match.txt", "rb") as fp:   # Unpickling
   df = pickle.load(fp)

with open("list_matches.txt", "rb") as fp:   # Unpickling
   df2 = pickle.load(fp)

with open("heroes.txt", "rb") as fp:   #Pickling
    df_heroes = pickle.load(fp)

In [9]:
with open("perfil.txt", "rb") as fp:   #Pickling
    perfil = pickle.load(fp)

In [31]:
df3 = pd.DataFrame(perfil)
df3

,tracked_until,solo_competitive_rank,competitive_rank,profile,rank_tier,leaderboard_rank,mmr_estimate
account_id,1610026887,3008,None,130441299,51,None,NaN
avatar,1610026887,3008,None,https://steamcdn-a.akamaihd.net/steamcommunity...,51,None,NaN
avatarfull,1610026887,3008,None,https://steamcdn-a.akamaihd.net/steamcommunity...,51,None,NaN
avatarmedium,1610026887,3008,None,https://steamcdn-a.akamaihd.net/steamcommunity...,51,None,NaN
cheese,1610026887,3008,None,0,51,None,NaN
estimate,1610026887,3008,None,NaN,51,None,3118.0
is_contributor,1610026887,3008,None,False,51,None,NaN
last_login,1610026887,3008,None,2020-10-29T02:21:17.294Z,51,None,NaN
loccountrycode,1610026887,3008,None,None,51,None,NaN
name,1610026887,3008,None,None,51,None,NaN


In [114]:
r = pd.DataFrame(df3['rank_tier'].transpose())
p = pd.DataFrame(df3['profile'].transpose())
m = pd.DataFrame(df3['mmr_estimate'].transpose())

In [115]:
p.loc['rank_tier'] = [r['rank_tier'][0]]

In [117]:
p.loc['mmr_estimate'] = [m['mmr_estimate']['estimate']]

In [120]:
p = p.transpose()

In [9]:
#Limpando os registros sem lista de players - provavelmente o jogo nao carregou ou os dados já se perderam
df = pd.DataFrame(df)
df = df[~df['players'].isnull()].reset_index()
df = df[['duration','first_blood_time', 'match_id', 'patch', 'players']]

df_heroes = pd.DataFrame(df_heroes )
df_heroes = df_heroes[['agi_gain', 'attack_range', 'attack_rate','attack_type', 'base_agi', 'base_armor', 'base_attack_max',
                      'base_attack_min', 'base_health', 'base_health_regen', 'base_int','base_mana', 'base_mana_regen', 'base_str',
                      'hero_id', 'icon', 'id', 'img', 'int_gain', 'localized_name','move_speed', 'name', 'primary_attr','projectile_speed', 'roles','str_gain']]

In [10]:
pi= []
#para cada partida que eu joguei
for i in range(0,len(df)):
    print(i)
    #print(df['players'][i])
    for m in range(0,len(df['players'][i])):
        if df['players'][i][m]['account_id']==id:
            print('kills: ',df['players'][i][m]['kills'])
            print(df['players'][i][m]['deaths'])
            print(df['players'][i][m]['assists'])
            print(df['players'][i][m]["hero_damage"])
            print(df['players'][i][m]["hero_healing"])
            print(df['players'][i][m]["gold_per_min"])
            print(df['players'][i][m]["xp_per_min"])
            print(df['players'][i][m]["last_hits"])
            print(df['players'][i][m]["denies"])
            print(df['players'][i][m]["level"])
            print(df['players'][i][m]["obs_placed"])
            print(df['players'][i][m]["sen_placed"])
            print(df['players'][i][m]["win"]) #0 false 1 positive
            print(df['players'][i][m]["lose"])#0 false 1 positive
            print(df['players'][i][m]["radiant_win"])
            print(df['players'][i][m]["isRadiant"])
            print(df['players'][i][m]["tower_damage"])
            print(df['players'][i][m]["total_gold"])
            
            pi.append([df['players'][i][m]['match_id'],
                df['players'][i][m]['player_slot'],
                df['players'][i][m]['hero_id'],
                df['players'][i][m]['kills'],
                df['players'][i][m]['deaths'],
                df['players'][i][m]['assists'],
                df['players'][i][m]["hero_damage"],
                df['players'][i][m]["hero_healing"],
                #df['players'][i][m]["purchase"],
                df['players'][i][m]["gold_per_min"],
                df['players'][i][m]["xp_per_min"],
                df['players'][i][m]["last_hits"],
                df['players'][i][m]["denies"],
                df['players'][i][m]["level"],
                df['players'][i][m]["obs_placed"],
                df['players'][i][m]["sen_placed"],
                df['players'][i][m]["win"],
                df['players'][i][m]["isRadiant"],
                df['players'][i][m]["radiant_win"],
                df['players'][i][m]["tower_damage"],
                df['players'][i][m]["total_gold"]])

0
kills:  3
6
9
10214
4292
274
336
24
1
14
7
14
1
0
False
False
494
8521
1
kills:  1
8
29
14715
0
241
500
24
3
21
7
17
0
1
True
False
0
10150
2
kills:  5
6
20
13508
0
356
552
42
0
24
9
25
1
0
False
False
216
16791
3
kills:  2
7
14
15214
4365
261
439
87
5
21
3
5
0
1
True
False
638
12567
4
kills:  6
12
26
25776
0
302
529
81
6
25
17
24
0
1
False
True
59
16358
5
kills:  1
9
17
7626
0
303
441
20
2
18
8
16
1
0
False
False
68
10387
6
kills:  1
6
24
11365
857
276
455
25
3
17
10
18
1
0
True
True
1025
8979
7
kills:  1
1
16
6626
0
279
333
13
1
12
8
10
1
0
True
True
1413
7100
8
kills:  7
1
22
12605
4033
329
530
77
6
21
4
3
1
0
True
True
1898
12644
9
kills:  3
2
9
6773
0
278
422
22
2
14
2
9
1
0
False
False
302
6574
10
kills:  4
8
15
7280
0
305
425
28
3
17
8
15
1
0
True
True
183
9836
11
kills:  2
0
19
6597
4901
320
505
30
5
17
6
9
1
0
False
False
1305
9450
12
kills:  2
2
14
16719
0
383
662
67
5
24
11
17
1
0
False
False
1102
15211
13
kills:  4
5
27
10268
0
275
489
45
5
21
17
20
1
0
False
False
480
11

0
319
499
84
2
25
None
None
1
0
True
True
704
18262
112
kills:  2
6
3
5660
0
214
262
33
1
12
None
None
0
1
True
False
0
5738
113
kills:  5
7
20
27775
0
313
416
40
1
19
None
None
0
1
False
True
1281
12963
114
kills:  2
7
14
6881
0
290
322
28
2
15
None
None
1
0
True
True
458
10386
115
kills:  3
1
16
12359
0
376
405
109
4
19
None
None
1
0
False
False
2004
15741
116
kills:  4
6
6
12137
0
299
340
38
3
14
None
None
0
1
True
False
0
8431
117
kills:  3
6
13
10720
0
277
321
40
5
15
9
13
0
1
True
False
115
9431
118
kills:  3
7
11
9435
0
206
224
43
3
13
None
None
0
1
True
False
0
7659
119
kills:  5
0
22
13131
0
448
441
75
8
17
None
None
1
0
False
False
467
13693
120
kills:  1
9
8
8444
0
171
248
46
0
14
None
None
0
1
True
False
104
7116
121
kills:  2
13
10
6197
0
183
287
17
1
14
13
15
0
1
True
False
60
6429
122
kills:  7
15
13
16479
1449
292
325
35
1
17
None
None
0
1
False
True
509
12512
123
kills:  6
6
27
18330
0
324
475
28
0
19
12
16
1
0
False
False
1151
11728
124
kills:  4
5
18
9029
0
300
430
1

15
26
1
0
True
True
1197
23474
222
kills:  12
4
17
18544
4718
450
593
84
6
22
None
None
1
0
False
False
2671
16627
223
kills:  13
2
15
15804
1773
407
489
29
0
17
None
None
1
0
True
True
773
11592
224
kills:  5
6
24
10944
0
444
439
117
3
17
None
None
1
0
True
True
3624
14156
225
kills:  2
15
17
20549
4954
323
574
173
0
25
None
None
0
1
False
True
976
17285
226
kills:  6
5
17
17405
0
388
567
106
1
23
None
None
1
0
False
False
807
17266
227
kills:  5
3
22
10057
480
358
408
22
1
15
None
None
1
0
True
True
2017
9546
228
kills:  7
1
15
13259
0
470
583
20
2
18
None
None
1
0
True
True
967
11656
229
kills:  6
5
20
None
None
343
494
59
5
21
None
None
1
0
False
False
None
13971
230
kills:  12
7
28
41759
0
582
765
386
6
25
None
None
1
0
True
True
3660
32504
231
kills:  7
7
29
25385
3752
415
769
79
2
25
9
11
1
0
True
True
3572
17879
232
kills:  5
13
8
20324
146
327
418
54
6
19
None
None
0
1
False
True
0
13516
233
kills:  12
7
32
27231
0
412
565
281
6
25
None
None
0
1
True
False
2324
26951
234
kills

25
10
25
1
0
True
True
950
23195
347
kills:  5
4
23
28564
3996
388
668
103
3
25
None
None
1
0
False
False
1267
17893
348
kills:  10
11
23
58679
4772
636
797
385
5
25
None
None
0
1
False
True
2428
41361
349
kills:  7
5
18
15588
3905
364
586
73
3
24
None
None
1
0
True
True
1155
15724
350
kills:  6
13
31
None
None
342
470
58
5
25
None
None
1
0
False
False
None
20337
351
kills:  5
6
17
5376
5424
282
285
39
0
14
None
None
1
0
False
False
430
9724
352
kills:  0
6
2
10064
0
169
202
35
1
10
None
None
0
1
False
True
0
4760
353
kills:  3
2
12
None
None
292
429
109
2
20
None
None
0
1
False
True
None
12215
354
kills:  16
6
26
24006
0
522
679
103
4
25
None
None
1
0
True
True
4887
22167
355
kills:  3
7
10
19549
177
406
538
172
4
22
None
None
0
1
True
False
175
15976
356
kills:  3
6
18
17160
2248
411
607
229
4
25
None
None
0
1
False
True
314
20056
357
kills:  8
9
22
20848
0
435
540
151
4
25
None
None
1
0
True
True
2705
22431
358
kills:  5
2
30
16704
0
500
648
73
5
22
None
None
1
0
False
False
2074
16

2
22
None
None
1
0
True
True
131
15158
472
kills:  1
10
4
9878
0
172
289
14
0
13
None
None
0
1
True
False
0
4618
473
kills:  2
6
18
14288
0
312
436
78
6
19
None
None
1
0
False
False
434
11710
474
kills:  2
7
32
10859
0
273
382
22
4
16
None
None
1
0
True
True
1132
8799
475
kills:  1
4
21
6822
110
278
399
30
5
16
None
None
1
0
False
False
438
8599
476
kills:  5
6
16
14519
0
369
538
79
6
23
17
25
0
1
False
True
406
15498
477
kills:  4
7
20
11130
400
375
560
49
5
24
None
None
1
0
False
False
1646
17281
478
kills:  7
5
23
19522
0
455
642
84
2
24
None
None
1
0
False
False
755
19041
479
kills:  1
6
2
10623
0
183
293
29
2
13
None
None
0
1
False
True
364
5166
480
kills:  0
9
5
10666
0
208
265
29
4
11
None
None
0
1
False
True
0
4683
481
kills:  2
5
20
18088
0
393
622
141
6
23
None
None
1
0
False
False
2634
15307
482
kills:  4
6
7
8420
0
229
281
32
1
13
None
None
0
1
False
True
20
6759
483
kills:  3
12
8
18669
0
279
364
104
3
19
None
None
0
1
False
True
471
12834
484
kills:  5
5
26
10375
0
369
67

...

523
86
4
17
None
None
1
0
False
False
None
15277
3521
kills:  5
7
17
11985
0
286
371
47
7
18
None
None
1
0
False
False
918
14309
3522
kills:  7
4
10
9795
0
244
373
40
4
16
None
None
0
1
True
False
78
9093
3523
kills:  7
5
8
15612
886
305
359
147
2
19
None
None
0
1
False
True
788
17324
3524
kills:  5
13
13
None
None
244
265
12
4
13
None
None
0
1
False
True
None
8324
3525
kills:  12
3
33
None
None
479
669
107
4
25
None
None
1
0
False
False
None
23167
3526
kills:  7
12
19
None
None
278
364
89
3
20
None
None
0
1
False
True
None
17444
3527
kills:  7
4
9
None
None
284
460
43
4
16
None
None
0
1
False
True
None
8827
3528
kills:  12
0
7
None
None
613
714
164
5
22
None
None
1
0
False
False
None
23498
3529
kills:  3
4
14
None
None
327
457
149
2
20
None
None
0
1
True
False
None
15854
3530
kills:  15
1
12
None
None
601
591
159
1
20
None
None
1
0
True
True
None
22807
3531
kills:  1
8
9
None
None
255
279
74
2
14
None
None
0
1
False
True
None
9481
3532
kills:  4
6
9
5475
0
232
311
36
3
15
None
None
0


433
588
196
5
21
None
None
1
0
True
True
None
18409
3671
kills:  14
5
13
None
None
535
655
192
27
24
None
None
1
0
False
False
None
24404
3672
kills:  4
3
8
None
None
699
600
219
5
20
None
None
1
0
True
True
None
26631
3673
kills:  1
7
2
None
None
156
204
8
5
10
None
None
0
1
True
False
None
4508
3674
kills:  5
7
9
None
None
302
509
78
12
17
None
None
0
1
False
True
None
9815
3675
kills:  2
4
11
None
None
327
391
44
6
14
None
None
1
0
False
False
None
9820
3676
kills:  12
2
5
None
None
561
589
150
13
20
None
None
1
0
False
False
None
20420
3677
kills:  3
7
10
None
None
278
342
82
2
15
None
None
0
1
True
False
None
9776
3678
kills:  6
5
6
None
None
226
343
20
7
16
None
None
0
1
False
True
None
9145
3679
kills:  9
4
1
None
None
371
493
65
14
15
None
None
0
1
True
False
None
9571
3680
kills:  2
5
3
None
None
360
325
82
3
11
None
None
0
1
False
True
None
8418
3681
kills:  7
10
13
None
None
315
419
143
9
22
None
None
0
1
False
True
None
20233
3682
kills:  3
10
7
None
None
268
274
41
1
15
No

None
517
504
223
1
19
None
None
0
1
False
True
None
20266
3796
kills:  0
0
0
None
None
395
436
39
14
8
None
None
1
0
True
True
None
3265
3797
kills:  16
7
8
None
None
437
472
143
2
19
None
None
0
1
True
False
None
18354
3798
kills:  7
1
19
17599
0
537
632
285
8
24
None
None
1
0
False
False
3120
26760
3799
kills:  6
4
20
13141
0
398
446
82
2
18
None
None
1
0
True
True
130
15601
3800
kills:  14
4
17
None
None
385
606
99
1
25
None
None
1
0
False
False
None
20571
3801
kills:  1
3
4
None
None
262
288
57
0
12
None
None
0
1
True
False
None
7763
3802
kills:  3
7
6
None
None
253
294
71
2
13
None
None
0
1
True
False
None
7775
3803
kills:  3
7
18
None
None
289
389
63
7
16
None
None
0
1
False
True
None
10423
3804
kills:  2
8
7
None
None
225
260
46
2
13
None
None
0
1
True
False
None
7950
3805
kills:  2
3
5
None
None
448
518
178
0
20
None
None
1
0
False
False
None
18823
3806
kills:  3
8
5
None
None
240
278
49
1
13
None
None
0
1
True
False
None
8212
3807
kills:  3
3
10
None
None
310
336
27
1
15
None


None
None
290
462
88
1
23
None
None
1
0
False
False
None
18390
3946
kills:  8
9
10
None
None
286
377
49
3
16
None
None
0
1
True
False
None
10567
3947
kills:  9
7
14
None
None
306
488
133
1
21
None
None
0
1
True
False
None
14591
3948
kills:  12
6
10
None
None
479
550
271
5
25
None
None
1
0
False
False
None
28189
3949
kills:  23
4
19
None
None
570
686
160
3
25
None
None
1
0
False
False
None
27065
3950
kills:  3
2
8
None
None
359
434
40
3
15
None
None
1
0
True
True
None
10758
3951
kills:  4
5
2
None
None
262
362
58
7
14
None
None
0
1
True
False
None
7873
3952
kills:  11
8
16
None
None
548
600
155
5
21
None
None
1
0
False
False
None
22915
3953
kills:  2
8
18
None
None
330
369
105
2
16
None
None
0
1
True
False
None
13508
3954
kills:  3
1
9
None
None
439
419
94
8
14
None
None
1
0
False
False
None
12431
3955
kills:  18
1
13
None
None
618
684
215
3
23
None
None
1
0
False
False
None
24905
3956
kills:  7
4
6
None
None
285
389
107
4
18
None
None
0
1
False
True
None
12825
3957
kills:  7
12
13
None

6
None
None
401
347
76
4
12
None
None
1
0
False
False
None
9035
4096
kills:  30
3
7
27390
0
639
715
195
5
25
None
None
1
0
True
True
7100
29809
4097
kills:  13
9
10
None
None
354
503
123
6
19
None
None
0
1
True
False
None
14342
4098
kills:  5
4
11
None
None
348
489
160
4
22
None
None
1
0
False
False
None
18536
4099
kills:  29
3
22
None
None
464
502
235
0
25
None
None
0
1
True
False
None
29943
4100
kills:  13
3
4
None
None
556
654
99
3
18
None
None
1
0
False
False
None
15799


In [11]:
df_perf = pd.DataFrame(pi, columns=['match_id','player_slot','hero_id',"kills","deaths",'assists',"hero_damage","hero_healing","gold_per_min","xp_per_min","last_hits",
                              "denies","level","obs_placed","sen_placed", "win", "isRadiant", "radiant_win", "tower_damage","total_gold"])
df_perf 

,match_id,player_slot,hero_id,kills,deaths,assists,hero_damage,hero_healing,gold_per_min,xp_per_min,last_hits,denies,level,obs_placed,sen_placed,win,isRadiant,radiant_win,tower_damage,total_gold
0,5715331723,132,112,3,6,9,10214.0,4292.0,274,336,24,1,14,7.0,14.0,1,False,False,494.0,8521
1,5713988500,132,87,1,8,29,14715.0,0.0,241,500,24,3,21,7.0,17.0,0,False,True,0.0,10150
2,5712585477,131,87,5,6,20,13508.0,0.0,356,552,42,0,24,9.0,25.0,1,False,False,216.0,16791
3,5712427786,128,9,2,7,14,15214.0,4365.0,261,439,87,5,21,3.0,5.0,0,False,True,638.0,12567
4,5711175066,2,87,6,12,26,25776.0,0.0,302,529,81,6,25,17.0,24.0,0,True,False,59.0,16358
5,5711100129,130,87,1,9,17,7626.0,0.0,303,441,20,2,18,8.0,16.0,1,False,False,68.0,10387
6,5711057728,1,68,1,6,24,11365.0,857.0,276,455,25,3,17,10.0,18.0,1,True,True,1025.0,8979
7,5710992268,3,87,1,1,16,6626.0,0.0,279,333,13,1,12,8.0,10.0,1,True,True,1413.0,7100
8,5710917619,1,9,7,1,22,12605.0,4033.0,329,530,77,6,21,4.0,3.0,1,True,True,1898.0,12644
9,5710869197,131,5,3,2,9,6773.0,0.0,278,422,22,2,14,2.0,9.0,1,False,False,302.0,6574


In [12]:
np.where(df_perf['isRadiant'], 'Radiant', 'Diretide')

array(['Diretide', 'Diretide', 'Diretide', ..., 'Diretide', 'Diretide',
       'Diretide'], dtype='<U8')

In [13]:
df_perf['Side'] = np.where(df_perf['isRadiant'], 'Radiant', 'Diretide')
df_perf['Result'] = df_perf['win'].replace({1:'Win', 0:'Loss'})

In [14]:
df_perf.drop(['win','isRadiant','radiant_win'],axis=1,inplace=True)

In [15]:
df_perf

,match_id,player_slot,hero_id,kills,deaths,assists,hero_damage,hero_healing,gold_per_min,xp_per_min,last_hits,denies,level,obs_placed,sen_placed,tower_damage,total_gold,Side,Result
0,5715331723,132,112,3,6,9,10214.0,4292.0,274,336,24,1,14,7.0,14.0,494.0,8521,Diretide,Win
1,5713988500,132,87,1,8,29,14715.0,0.0,241,500,24,3,21,7.0,17.0,0.0,10150,Diretide,Loss
2,5712585477,131,87,5,6,20,13508.0,0.0,356,552,42,0,24,9.0,25.0,216.0,16791,Diretide,Win
3,5712427786,128,9,2,7,14,15214.0,4365.0,261,439,87,5,21,3.0,5.0,638.0,12567,Diretide,Loss
4,5711175066,2,87,6,12,26,25776.0,0.0,302,529,81,6,25,17.0,24.0,59.0,16358,Radiant,Loss
5,5711100129,130,87,1,9,17,7626.0,0.0,303,441,20,2,18,8.0,16.0,68.0,10387,Diretide,Win
6,5711057728,1,68,1,6,24,11365.0,857.0,276,455,25,3,17,10.0,18.0,1025.0,8979,Radiant,Win
7,5710992268,3,87,1,1,16,6626.0,0.0,279,333,13,1,12,8.0,10.0,1413.0,7100,Radiant,Win
8,5710917619,1,9,7,1,22,12605.0,4033.0,329,530,77,6,21,4.0,3.0,1898.0,12644,Radiant,Win
9,5710869197,131,5,3,2,9,6773.0,0.0,278,422,22,2,14,2.0,9.0,302.0,6574,Diretide,Win


In [12]:
for i in range(0,len(df_heroes)):
    df_heroes['img'][i] = "https://api.opendota.com"+df_heroes['img'][i]
    df_heroes['icon'][i] = "https://api.opendota.com"+df_heroes['icon'][i]

C:\Users\lucaskenjis\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\lucaskenjis\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
df_perf.to_csv("performance.csv")

In [14]:
df_heroes.to_csv('heroes.csv')

In [121]:
p.to_csv('perfil.csv')